# Dataset com dados de SP para balanceamento

Este notebook se refere apenas as etapas de pré-processamento realizadas na Iniciação Científica

In [ ]:
# Importações de Bibliotecas
import pandas as pd
import numpy as np
import sklearn
import math

# Datas
from datetime import datetime, timedelta

In [ ]:
# Local do Dataset
url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2022/INFLUD22-23-01-2023.csv"

In [ ]:
colunas_desejadas = ["CLASSI_FIN", "SG_UF_NOT","DT_NOTIFIC", "DT_SIN_PRI", "CS_SEXO", "NU_IDADE_N",
                     "CS_GESTANT", "NOSOCOMIAL", "FEBRE", "TOSSE", "GARGANTA",
                     "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO",
                     "OUTRO_SIN", "FATOR_RISC", "CARDIOPATI", "DIABETES",
                     "OUT_MORBI", "ANTIVIRAL", "HOSPITAL", "DT_INTERNA", "UTI", "SUPORT_VEN",
                     "RAIOX_RES", "AMOSTRA", "TP_AMOSTRA", "PCR_RESUL", "DT_PCR",
                     "POS_PCROUT", "CRITERIO", "EVOLUCAO", "DT_EVOLUCA", "DOR_ABD",
                     "FADIGA", "PERD_OLFT", "PERD_PALA", "TOMO_RES", "RES_AN",
                     "VACINA_COV", "DOSE_1_COV", "DOSE_2_COV"]

In [ ]:
# Carregando o conjunto de dados com colunas específicas
data = pd.read_csv(url, sep=';', na_filter=True, usecols=colunas_desejadas)
print(f"{len(data)} linhas lidas")

561242 linhas lidas


In [ ]:
# Formato do Dataset
data.shape

(561242, 44)

# Delimitando Dataset

Este dataset é composto por notificações realacionadas as Síndromes Respiratórias Agudas Graves SRAGs no Brasil no ano de 2022

In [ ]:
# Delimitando apenas para notificações de Covid-19
data = data[data['CLASSI_FIN'] == 5]
data.shape

(235891, 44)

In [ ]:
# Deixando apenas com notificações de São Paulo
data = data[data['SG_UF_NOT'] == 'SP']
data.shape

(74782, 44)

In [ ]:
# Delimitando apenas para óbitos
data = data[(data['EVOLUCAO'] == 2)]

# Atualizando o target 'EVOLUCAO' de 2 para 1
data['EVOLUCAO'] = data['EVOLUCAO'].map({2: 1})
data.shape

(18710, 44)

In [ ]:
# Dados redundantes
data.drop(['CLASSI_FIN', 'SG_UF_NOT'], axis=1, inplace=True)
data.shape

(18710, 42)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18710 entries, 16 to 561223
Data columns (total 42 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  18710 non-null  object 
 1   DT_SIN_PRI  18710 non-null  object 
 2   CS_SEXO     18710 non-null  object 
 3   NU_IDADE_N  18710 non-null  int64  
 4   CS_GESTANT  18710 non-null  int64  
 5   NOSOCOMIAL  16642 non-null  float64
 6   FEBRE       14585 non-null  float64
 7   TOSSE       15684 non-null  float64
 8   GARGANTA    12980 non-null  float64
 9   DISPNEIA    16070 non-null  float64
 10  DESC_RESP   15107 non-null  float64
 11  SATURACAO   15987 non-null  float64
 12  DIARREIA    12857 non-null  float64
 13  VOMITO      12765 non-null  float64
 14  OUTRO_SIN   13341 non-null  float64
 15  FATOR_RISC  18710 non-null  int64  
 16  CARDIOPATI  12042 non-null  float64
 17  DIABETES    11147 non-null  float64
 18  OUT_MORBI   11740 non-null  float64
 19  ANTIVIRAL   16197 non-n

# Tratamento de Valores Ignorados

In [ ]:
# Alteração dos valores 9 (ignorados) para NaN
for column in data:
  if ((data[column].dtypes != 'object') and
   (column != 'NU_IDADE_N')) or (column == 'CS_SEXO'):
    data[column] = data[column].apply(lambda x: np.nan if x == 9 else x)

# Filtro de 90% em linhas.

In [ ]:
# Limiar de procentagem de dados faltantes
threshold = 0.1

# Calcula a porcentagem de dados faltantes em cada linha
porcentagem_nan = data.isnull().sum(axis=1) / data.shape[1]

# Filtra as linhas que têm mais de 50% dos dados faltantes
linhas_remocao = porcentagem_nan[porcentagem_nan > threshold].index

# Remove as linhas
data = data.drop(linhas_remocao, axis=0)

print(f'{len(linhas_remocao)} linhas foram removidas')
print(data.shape)

10932 linhas foram removidas
(7778, 42)


In [ ]:
# Reinicia o index de cada instância
data = data.reset_index(drop=True)

# Preenchimento de valores NaN do tipo float.

In [ ]:
# Dados faltantes
faltantes = data.isnull().sum()
faltantes

DT_NOTIFIC       0
DT_SIN_PRI       0
CS_SEXO          0
NU_IDADE_N       0
CS_GESTANT     107
NOSOCOMIAL     245
FEBRE           30
TOSSE            5
GARGANTA        44
DISPNEIA         7
DESC_RESP       26
SATURACAO       22
DIARREIA        22
VOMITO          15
OUTRO_SIN      404
FATOR_RISC       0
CARDIOPATI     482
DIABETES       521
OUT_MORBI      740
ANTIVIRAL      899
HOSPITAL         2
DT_INTERNA     213
UTI            272
SUPORT_VEN     144
RAIOX_RES     2177
AMOSTRA          0
TP_AMOSTRA      43
PCR_RESUL      169
DT_PCR        2859
POS_PCROUT    3251
CRITERIO        31
EVOLUCAO         0
DT_EVOLUCA       3
DOR_ABD         17
FADIGA          13
PERD_OLFT       23
PERD_PALA       25
TOMO_RES      1667
RES_AN         399
VACINA_COV      38
DOSE_1_COV    1074
DOSE_2_COV    1170
dtype: int64

Todos os NaN serão substituidos pelo valor da mediana daquele atributo.

In [ ]:
# Seleciona as colunas do tipo float
colunas_float = data.select_dtypes(include='float64').columns

# Substitui os valores NaN pela mediana de cada coluna
data[colunas_float] = data[colunas_float].fillna(data[colunas_float].median().iloc[0])

In [ ]:
# Dados faltantes
faltantes = data.isnull().sum()
faltantes

DT_NOTIFIC       0
DT_SIN_PRI       0
CS_SEXO          0
NU_IDADE_N       0
CS_GESTANT       0
NOSOCOMIAL       0
FEBRE            0
TOSSE            0
GARGANTA         0
DISPNEIA         0
DESC_RESP        0
SATURACAO        0
DIARREIA         0
VOMITO           0
OUTRO_SIN        0
FATOR_RISC       0
CARDIOPATI       0
DIABETES         0
OUT_MORBI        0
ANTIVIRAL        0
HOSPITAL         0
DT_INTERNA     213
UTI              0
SUPORT_VEN       0
RAIOX_RES        0
AMOSTRA          0
TP_AMOSTRA       0
PCR_RESUL        0
DT_PCR        2859
POS_PCROUT       0
CRITERIO         0
EVOLUCAO         0
DT_EVOLUCA       3
DOR_ABD          0
FADIGA           0
PERD_OLFT        0
PERD_PALA        0
TOMO_RES         0
RES_AN           0
VACINA_COV       0
DOSE_1_COV    1074
DOSE_2_COV    1170
dtype: int64

# Tratando Datas

Criação de uma coluna chamada "QTD_DIAS" que é resultante da relação entre data da Notificação do paciente até a Evolução.

*DT_NOTIFIC com DT_EVOLUCA*

In [ ]:
# QTD_DIAS armazena a quantidade de dias desde a notificação até a evolução.
data['QTD_DIAS'] = ''

In [ ]:
# Calcula o intervalo de dias entre duas datas
def calcular_qtd_dias(row):
  if pd.isna(row['DT_EVOLUCA']):
    return np.nan
  else:
    data1 = datetime.strptime(row['DT_NOTIFIC'], '%d/%m/%Y')
    data2 = datetime.strptime(row['DT_EVOLUCA'], '%d/%m/%Y')
    dias = abs((data2 - data1).days)
    return dias

# Chama a função e atribui o resultado à coluna 'QTD_DIAS'
data['QTD_DIAS'] = data.apply(calcular_qtd_dias, axis=1)

In [ ]:
data[['DT_NOTIFIC', 'DT_EVOLUCA', 'QTD_DIAS']]

,DT_NOTIFIC,DT_EVOLUCA,QTD_DIAS
0,06/01/2022,15/01/2022,9.0
1,17/01/2022,18/01/2022,1.0
2,20/01/2022,08/02/2022,19.0
3,20/01/2022,21/01/2022,1.0
4,19/01/2022,26/01/2022,7.0
...,...,...,...
7773,20/07/2022,22/07/2022,2.0
7774,27/06/2022,20/06/2022,7.0
7775,20/07/2022,20/07/2022,0.0
7776,01/12/2022,03/12/2022,2.0


In [ ]:
data['QTD_DIAS'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: QTD_DIAS
Non-Null Count  Dtype  
--------------  -----  
7775 non-null   float64
dtypes: float64(1)
memory usage: 60.9 KB


In [ ]:
# Calcula a mediana de 'QTD_DIAS'
data['QTD_DIAS'].median()

6.0

In [ ]:
# Conta os valores NaN em 'QTD_DIAS'
data['QTD_DIAS'].isnull().sum()

3

In [ ]:
# Preenche valores NaN com a mediana
data['QTD_DIAS'].fillna(data['QTD_DIAS'].median(), inplace=True)

In [ ]:
# Converte o tipo de dados de float para int
data['QTD_DIAS'] = data['QTD_DIAS'].astype(int)

In [ ]:
data['QTD_DIAS'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: QTD_DIAS
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


Criação de uma coluna chamada "DIAS_INTERNA" que representa a quantidade de dias que um paciente ficou hospitalizado.

*DT_INTERNA com DT_EVOLUCA*

In [ ]:
# DIAS_INTERNA armazena a quantidade de dias internado desde a data de internação
# até a evolução.
data['DIAS_INTERNA'] = ''

In [ ]:
# Calcula o intervalo de dias entre duas datas
def calc_dias_interna(row):
  if row['HOSPITAL'] == 1:
    if pd.isna(row['DT_INTERNA']) or pd.isna(row['DT_EVOLUCA']):
      return np.nan
    else:
      data1 = datetime.strptime(row['DT_INTERNA'], '%d/%m/%Y')
      data2 = datetime.strptime(row['DT_EVOLUCA'], '%d/%m/%Y')
      return abs((data2 - data1).days)
  else:
    return 0

# Chama a função e atribui o resultado à coluna 'DIAS_INTERNA'
data['DIAS_INTERNA'] = data.apply(calc_dias_interna, axis=1)

In [ ]:
data[['DT_INTERNA', 'DT_EVOLUCA', 'DIAS_INTERNA']]

,DT_INTERNA,DT_EVOLUCA,DIAS_INTERNA
0,06/01/2022,15/01/2022,9.0
1,17/01/2022,18/01/2022,1.0
2,15/01/2022,08/02/2022,24.0
3,20/01/2022,21/01/2022,1.0
4,19/01/2022,26/01/2022,7.0
...,...,...,...
7773,21/07/2022,22/07/2022,1.0
7774,15/06/2022,20/06/2022,5.0
7775,20/06/2022,20/07/2022,30.0
7776,01/12/2022,03/12/2022,2.0


In [ ]:
data['DIAS_INTERNA'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: DIAS_INTERNA
Non-Null Count  Dtype  
--------------  -----  
7752 non-null   float64
dtypes: float64(1)
memory usage: 60.9 KB


In [ ]:
# Calcula a mediana de 'DIAS_INTERNA'
data['DIAS_INTERNA'].median()

8.0

In [ ]:
# Conta os valores NaN em 'DIAS_INTERNA'
data['DIAS_INTERNA'].isnull().sum()

26

In [ ]:
# Preenche valores NaN com a mediana
data['DIAS_INTERNA'].fillna(data['DIAS_INTERNA'].median(), inplace=True)

In [ ]:
# Converte o tipo de dados de float para int
data['DIAS_INTERNA'] = data['DIAS_INTERNA'].astype(int)

In [ ]:
data['DIAS_INTERNA'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: DIAS_INTERNA
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


In [ ]:
data[['DT_INTERNA', 'DT_EVOLUCA', 'DT_SIN_PRI', 'DT_NOTIFIC', 'DT_PCR']]

,DT_INTERNA,DT_EVOLUCA,DT_SIN_PRI,DT_NOTIFIC,DT_PCR
0,06/01/2022,15/01/2022,05/01/2022,06/01/2022,10/01/2022
1,17/01/2022,18/01/2022,02/01/2022,17/01/2022,17/01/2022
2,15/01/2022,08/02/2022,14/01/2022,20/01/2022,16/01/2022
3,20/01/2022,21/01/2022,12/01/2022,20/01/2022,22/01/2022
4,19/01/2022,26/01/2022,15/01/2022,19/01/2022,22/01/2022
...,...,...,...,...,...
7773,21/07/2022,22/07/2022,09/07/2022,20/07/2022,22/07/2022
7774,15/06/2022,20/06/2022,14/06/2022,27/06/2022,NaN
7775,20/06/2022,20/07/2022,19/06/2022,20/07/2022,NaN
7776,01/12/2022,03/12/2022,26/11/2022,01/12/2022,NaN


Criação de uma coluna "SINT_ATE_NOTIF" que representa a quantidade de dias entre os primeiros sintomas do paciente até a notificação.

*DT_SIN_PRI com DT_NOTIFIC*

In [ ]:
data['SINT_ATE_NOTIF'] = ''

In [ ]:
# Calcula o intervalo de dias entre duas datas
def calc_dias_sintomas(row):
  data1 = datetime.strptime(row['DT_SIN_PRI'], '%d/%m/%Y')
  data2 = datetime.strptime(row['DT_NOTIFIC'], '%d/%m/%Y')
  return abs((data2 - data1).days)

# Chama a função e atribui o resultado à coluna 'SINT_ATE_NOTIF'
data['SINT_ATE_NOTIF'] = data.apply(calc_dias_sintomas, axis=1)

In [ ]:
data[['DT_SIN_PRI', 'DT_NOTIFIC', 'SINT_ATE_NOTIF']]

,DT_SIN_PRI,DT_NOTIFIC,SINT_ATE_NOTIF
0,05/01/2022,06/01/2022,1
1,02/01/2022,17/01/2022,15
2,14/01/2022,20/01/2022,6
3,12/01/2022,20/01/2022,8
4,15/01/2022,19/01/2022,4
...,...,...,...
7773,09/07/2022,20/07/2022,11
7774,14/06/2022,27/06/2022,13
7775,19/06/2022,20/07/2022,31
7776,26/11/2022,01/12/2022,5


In [ ]:
data['SINT_ATE_NOTIF'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: SINT_ATE_NOTIF
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


Criação da coluna "PCR_EVOLUCAO" que calcula a diferença de dias entre a data do PCR e a data da evolução.

*DT_PCR com DT_EVOLUCA*

In [ ]:
data['PCR_EVOLUCAO'] = ''

In [ ]:
# Calcula a diferença de dias entre duas datas
def calc_pcr_evo(row):
  if row['PCR_RESUL'] != 4:
    if pd.isna(row['DT_PCR']) or pd.isna(row['DT_EVOLUCA']):
      return np.nan
    else:
      data1 = datetime.strptime(row['DT_PCR'], '%d/%m/%Y')
      data2 = datetime.strptime(row['DT_EVOLUCA'], '%d/%m/%Y')
      return abs((data2 - data1).days)
  else:
    return 0

# Chama a função e atribui o resultado à coluna 'PCR_EVOLUCAO'
data['PCR_EVOLUCAO'] = data.apply(calc_pcr_evo, axis=1)

In [ ]:
data[['DT_PCR', 'DT_EVOLUCA', 'PCR_EVOLUCAO']]

,DT_PCR,DT_EVOLUCA,PCR_EVOLUCAO
0,10/01/2022,15/01/2022,5.0
1,17/01/2022,18/01/2022,1.0
2,16/01/2022,08/02/2022,23.0
3,22/01/2022,21/01/2022,1.0
4,22/01/2022,26/01/2022,4.0
...,...,...,...
7773,22/07/2022,22/07/2022,0.0
7774,NaN,20/06/2022,0.0
7775,NaN,20/07/2022,0.0
7776,NaN,03/12/2022,NaN


In [ ]:
data['PCR_EVOLUCAO'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: PCR_EVOLUCAO
Non-Null Count  Dtype  
--------------  -----  
6831 non-null   float64
dtypes: float64(1)
memory usage: 60.9 KB


In [ ]:
# Calcula a mediana de 'PCR_EVOLUCAO'
data['PCR_EVOLUCAO'].median()

3.0

In [ ]:
# Conta os valores NaN em 'PCR_EVOLUCAO'
data['PCR_EVOLUCAO'].isnull().sum()

947

In [ ]:
# Preenche valores NaN com a mediana
data['PCR_EVOLUCAO'].fillna(data['PCR_EVOLUCAO'].median(), inplace=True)

In [ ]:
# Converte o tipo de dados de float para int
data['PCR_EVOLUCAO'] = data['PCR_EVOLUCAO'].astype(int)

In [ ]:
data['PCR_EVOLUCAO'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: PCR_EVOLUCAO
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


Segunda Dose

Criação da coluna "DIAS_DOSE2" que calcula a quantidade de dias da data da segunda dose até a notificação.

*DOSE_2_COV com DT_NOTIFIC*

In [ ]:
data['DIAS_DOSE2'] = ''

In [ ]:
# Calcula o intervalo de dias entre duas datas
def vac_segunda_dose(row):
  if row['VACINA_COV'] == 1:
    if pd.isna(row['DOSE_2_COV']):
      return np.nan
    else:
      data1 = datetime.strptime(row['DOSE_2_COV'], '%d/%m/%Y')
      data2 = datetime.strptime(row['DT_NOTIFIC'], '%d/%m/%Y')
      return abs((data2 - data1).days)
  else:
    return 0

# Chama a função e atribui o resultado à coluna 'DIAS_DOSE2'
data['DIAS_DOSE2'] = data.apply(vac_segunda_dose, axis=1)

In [ ]:
data[['DOSE_1_COV','DOSE_2_COV', 'DT_NOTIFIC', 'DIAS_DOSE2']]

,DOSE_1_COV,DOSE_2_COV,DT_NOTIFIC,DIAS_DOSE2
0,03/03/2021,27/05/2021,06/01/2022,224.0
1,06/05/2021,23/11/2021,17/01/2022,55.0
2,NaN,NaN,20/01/2022,0.0
3,NaN,NaN,20/01/2022,0.0
4,07/05/2021,07/08/2021,19/01/2022,165.0
...,...,...,...,...
7773,19/03/2021,09/04/2021,20/07/2022,467.0
7774,20/02/2021,20/03/2021,27/06/2022,464.0
7775,15/02/2021,02/03/2021,20/07/2022,505.0
7776,13/12/2021,14/01/2022,01/12/2022,321.0


In [ ]:
data['DIAS_DOSE2'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: DIAS_DOSE2
Non-Null Count  Dtype  
--------------  -----  
7421 non-null   float64
dtypes: float64(1)
memory usage: 60.9 KB


In [ ]:
# Calcula a mediana
data['DIAS_DOSE2'].median()

315.0

In [ ]:
# Conta os valores NaN
data['DIAS_DOSE2'].isnull().sum()

357

In [ ]:
# Preenche valores NaN com a mediana
data['DIAS_DOSE2'].fillna(data['DIAS_DOSE2'].median(), inplace=True)

# Converte o tipo de dados de float para int
data['DIAS_DOSE2'] = data['DIAS_DOSE2'].astype(int)

In [ ]:
data['DIAS_DOSE2'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: DIAS_DOSE2
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


PRIMEIRA DOSE

Preenchimento dos dados

O simples preenchimento com a moda deixará alguns dados incoerentes, pois existem instâncias que possuem NaN no valor da data da primeira dose e possui a data da segunda dose. Com isso, pode ocorrer instâncias onde os dias da segunda dose serem mais distantes que os dias da primeira dose, causando incoerência.

Dessa forma, uma nova coluna será criada para extrair a mediana da diferença dos dias entre a segunda e a primeira dose.

In [ ]:
# Criação de uma coluna par armazenar a quantidade de dias entre a primeira e segunda dose
data['INTERVALO_VACINAS'] = ''

In [ ]:
# Calcula o intervalo entre vacinas
def calc_interval_vac(row):
  if pd.isna(row['DOSE_2_COV']) or pd.isna(row['DOSE_1_COV']):
    return np.nan
  else:
    data1 = datetime.strptime(row['DOSE_2_COV'], '%d/%m/%Y')
    data2 = datetime.strptime(row['DOSE_1_COV'], '%d/%m/%Y')
    return abs((data2 - data1).days)

# Chama a função e atribui o resultado à coluna 'INTERVALO_VACINAS'
data['INTERVALO_VACINAS'] = data.apply(calc_interval_vac, axis=1)

In [ ]:
data[['DOSE_1_COV','DOSE_2_COV', 'INTERVALO_VACINAS']]

,DOSE_1_COV,DOSE_2_COV,INTERVALO_VACINAS
0,03/03/2021,27/05/2021,85.0
1,06/05/2021,23/11/2021,201.0
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,07/05/2021,07/08/2021,92.0
...,...,...,...
7773,19/03/2021,09/04/2021,21.0
7774,20/02/2021,20/03/2021,28.0
7775,15/02/2021,02/03/2021,15.0
7776,13/12/2021,14/01/2022,32.0


In [ ]:
# Calcula mediana
mediana_intervalo = data['INTERVALO_VACINAS'].median()
mediana_intervalo

28.0

In [ ]:
# Remove coluna
data.drop(['INTERVALO_VACINAS'], axis=1, inplace=True)

In [ ]:
# Armazena a quantidade de dias da primeira dose até a notificação
data['DIAS_DOSE1'] = ''

In [ ]:
# Calcula a quantidade de dias da primeira dose até a notificação
def calc_dias_vac(row):
  if row['VACINA_COV'] != 1:
    return 0

  if pd.isna(row['DOSE_1_COV']) and pd.isna(row['DOSE_2_COV']):
    return np.nan

  if pd.isna(row['DOSE_1_COV']):
    dataDose2 = datetime.strptime(row['DOSE_2_COV'], '%d/%m/%Y')
    data1 = dataDose2 - timedelta(days=mediana_intervalo)
  else:
    data1 = datetime.strptime(row['DOSE_1_COV'], '%d/%m/%Y')

  data2 = datetime.strptime(row['DT_NOTIFIC'], '%d/%m/%Y')
  return abs((data2 - data1).days)

# Chama a função e atribui o resultado à coluna 'DIAS_DOSE1'
data['DIAS_DOSE1'] = data.apply(calc_dias_vac, axis=1)

In [ ]:
data[['DOSE_1_COV','DOSE_2_COV', 'DT_NOTIFIC', 'DIAS_DOSE1', 'DIAS_DOSE2']]

,DOSE_1_COV,DOSE_2_COV,DT_NOTIFIC,DIAS_DOSE1,DIAS_DOSE2
0,03/03/2021,27/05/2021,06/01/2022,309.0,224
1,06/05/2021,23/11/2021,17/01/2022,256.0,55
2,NaN,NaN,20/01/2022,0.0,0
3,NaN,NaN,20/01/2022,0.0,0
4,07/05/2021,07/08/2021,19/01/2022,257.0,165
...,...,...,...,...,...
7773,19/03/2021,09/04/2021,20/07/2022,488.0,467
7774,20/02/2021,20/03/2021,27/06/2022,492.0,464
7775,15/02/2021,02/03/2021,20/07/2022,520.0,505
7776,13/12/2021,14/01/2022,01/12/2022,353.0,321


In [ ]:
# Conta os valores NaN
data['DIAS_DOSE1'].isnull().sum()

140

In [ ]:
# Calcula a mediana
data['DIAS_DOSE1'].median()

348.0

In [ ]:
# Preenche valores NaN com a mediana
data['DIAS_DOSE1'].fillna(data['DIAS_DOSE1'].median(), inplace=True)

# Converte o tipo de dados de float para int
data['DIAS_DOSE1'] = data['DIAS_DOSE1'].astype(int)

In [ ]:
data['DIAS_DOSE1'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: DIAS_DOSE1
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


In [ ]:
data[['DOSE_1_COV','DOSE_2_COV', 'DT_NOTIFIC', 'DIAS_DOSE1','DIAS_DOSE2']]

,DOSE_1_COV,DOSE_2_COV,DT_NOTIFIC,DIAS_DOSE1,DIAS_DOSE2
0,03/03/2021,27/05/2021,06/01/2022,309,224
1,06/05/2021,23/11/2021,17/01/2022,256,55
2,NaN,NaN,20/01/2022,0,0
3,NaN,NaN,20/01/2022,0,0
4,07/05/2021,07/08/2021,19/01/2022,257,165
...,...,...,...,...,...
7773,19/03/2021,09/04/2021,20/07/2022,488,467
7774,20/02/2021,20/03/2021,27/06/2022,492,464
7775,15/02/2021,02/03/2021,20/07/2022,520,505
7776,13/12/2021,14/01/2022,01/12/2022,353,321


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7778 entries, 0 to 7777
Data columns (total 48 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DT_NOTIFIC      7778 non-null   object 
 1   DT_SIN_PRI      7778 non-null   object 
 2   CS_SEXO         7778 non-null   object 
 3   NU_IDADE_N      7778 non-null   int64  
 4   CS_GESTANT      7778 non-null   float64
 5   NOSOCOMIAL      7778 non-null   float64
 6   FEBRE           7778 non-null   float64
 7   TOSSE           7778 non-null   float64
 8   GARGANTA        7778 non-null   float64
 9   DISPNEIA        7778 non-null   float64
 10  DESC_RESP       7778 non-null   float64
 11  SATURACAO       7778 non-null   float64
 12  DIARREIA        7778 non-null   float64
 13  VOMITO          7778 non-null   float64
 14  OUTRO_SIN       7778 non-null   float64
 15  FATOR_RISC      7778 non-null   int64  
 16  CARDIOPATI      7778 non-null   float64
 17  DIABETES        7778 non-null   f

In [ ]:
# Remove os atributos com datas
data.drop(['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INTERNA', 'DT_PCR', 'DT_EVOLUCA', 'DOSE_1_COV', 'DOSE_2_COV'], axis = 1, inplace = True)
data.shape

(7778, 41)

# Conversão do tipo de dado

Alterando o atributo "CS_SEXO" de object para int

In [ ]:
# Teste Objects
for i, row in data.iterrows():
  if row['CS_SEXO'] == 'M':
    data.at[i, 'CS_SEXO'] = 1
  if row['CS_SEXO'] == 'F':
    data.at[i, 'CS_SEXO'] = 2

# Convertendo object -> int
data['CS_SEXO'] = data['CS_SEXO'].astype(int)

data['CS_SEXO'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7778 entries, 0 to 7777
Series name: CS_SEXO
Non-Null Count  Dtype
--------------  -----
7778 non-null   int64
dtypes: int64(1)
memory usage: 60.9 KB


In [ ]:
# Convertendo float -> int
for column in data:
  if data[column].dtypes == 'float64':
    data[column] = data[column].astype(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7778 entries, 0 to 7777
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   CS_SEXO         7778 non-null   int64
 1   NU_IDADE_N      7778 non-null   int64
 2   CS_GESTANT      7778 non-null   int64
 3   NOSOCOMIAL      7778 non-null   int64
 4   FEBRE           7778 non-null   int64
 5   TOSSE           7778 non-null   int64
 6   GARGANTA        7778 non-null   int64
 7   DISPNEIA        7778 non-null   int64
 8   DESC_RESP       7778 non-null   int64
 9   SATURACAO       7778 non-null   int64
 10  DIARREIA        7778 non-null   int64
 11  VOMITO          7778 non-null   int64
 12  OUTRO_SIN       7778 non-null   int64
 13  FATOR_RISC      7778 non-null   int64
 14  CARDIOPATI      7778 non-null   int64
 15  DIABETES        7778 non-null   int64
 16  OUT_MORBI       7778 non-null   int64
 17  ANTIVIRAL       7778 non-null   int64
 18  HOSPITAL        7778 non-nul

In [ ]:
data.shape

(7778, 41)

In [ ]:
data.head()

,CS_SEXO,NU_IDADE_N,CS_GESTANT,NOSOCOMIAL,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,...,PERD_PALA,TOMO_RES,RES_AN,VACINA_COV,QTD_DIAS,DIAS_INTERNA,SINT_ATE_NOTIF,PCR_EVOLUCAO,DIAS_DOSE2,DIAS_DOSE1
0,2,83,5,2,2,1,2,2,1,2,...,2,6,1,1,9,9,1,5,224,309
1,1,61,6,2,2,2,2,2,1,1,...,2,6,1,1,1,1,15,1,55,256
2,1,79,6,2,2,1,2,1,1,1,...,2,1,6,2,19,24,6,23,0,0
3,1,78,6,2,1,1,1,1,1,1,...,2,6,4,2,1,1,8,1,0,0
4,1,62,6,2,2,2,2,1,2,1,...,2,6,6,1,7,7,4,4,165,257


# Gerando novo Dataset

In [ ]:
# Gerando um novo dataset
data.to_csv('dataset_Pre_Processado_SP_Obitos_Nao_Norm.csv', index=False)

In [ ]:
data['EVOLUCAO'].describe()

count    7778.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: EVOLUCAO, dtype: float64